<a href="https://colab.research.google.com/github/puja-maitra/NumerAI/blob/main/assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving correlations.npy to correlations (1).npy
Saving test.pic to test (1).pic
Saving train.pic to train (1).pic


In [5]:
import numpy as np
import pickle
import random
corr = None
ifTest = False
Ztr = {}
Zte = {}

# States are tuples of the form (x,y) where both x,y belong to the closed interval [0,9].

# The terminology for action is :
# -1 for SELL
# 0 for HOLD
# 1 for BUY

# The returnNext function would take in the state and action, and return the next state, which you would get alongwith the price change associated with this tuple of (state,action)

# This might be different in different turns as this change is stochastic in general with only a co-relation to the (state, action) tuple.


def returnNext(state, action):
    # Returns (priceChange, nextState) as a tuple
    #nextState is actually a 2-tuple representing the state
    a,b = state
    L = []
    if ifTest:
        L = Zte[(a,b)]
    else:
        L = Ztr[(a,b)]
    r = L[random.randint(0,len(L)-1)]
    pricc = None
    if action==-1:
        if r[0]==0:
            pricc =  r[4]
        elif r[0]==1:
            pricc =  r[3]
        else:
            pricc =  -r[3]
    elif action==1:
        if r[2]==0:
            pricc =  r[4]
        elif r[2]==1:
            pricc =  r[3]
        else:
            pricc =  -r[3]
    else:
        if r[1]==0:
            pricc =  r[4]
        elif r[1]==1:
            pricc =  r[3]
        else:
            pricc =  -r[3]
    num = 3*(10*a + b) + action + 1

    F = corr[num,:]
    ch = np.random.choice(np.arange(0,100),p = list(F))
    ca = ch // 10
    cb = ch % 10
    return (pricc,(ca,cb))



# Train your Markov Decision Process, make use of the returnNext function to model and act on the priceChanges dependence on the states and actions
def Train():

    #defining all states
    all_states = []   
    for i in range(10):
      for j in range(10):
        all_states.append((i,j))
    
    
    rewards = {}
    for i in range(10):
      for j in range(10):
        rewards[(i,j)] = i+j
    
    actions = {}
    for i in range(10):
      for j in range(10):
        actions[(i,j)] = (1,0,-1)
  

    #value iteration/policy part
    policy = {}
    for s in actions.keys():
      policy[s] = np.random.choice(1,0,-1)
    
    V = {}
    for s in all_states:
      if s in actions.keys():
          V[s] = 0
      if s ==(0,0):
          V[s]=-1
      if s == (9,9):
          V[s]=1
    


    iteration = 0
    while True:
        biggest_change = 0
        for s in all_states:            
            if s in policy:
                
                old_v = V[s]
                new_v = 0
                
                for a in actions[s]:
                    #if a == 1:
                    pc, nxt = returnNext(s,a)
                    
                    #Choose a new random action to do (transition probability)
                    random_1=np.random.choice([i for i in actions[s] if i != a])
                    #if random_1 == 1:
                    pc1, nxt1 = returnNext(s,random_1)
                
                    #Calculate the value
                    nxt = tuple(nxt)
                    nxt1 = tuple(nxt1)
                    GAMMA = 0.9
                    NOISE = 0.1
                    v = pc1 + (GAMMA * ((1-NOISE)* V[nxt] + (NOISE * V[nxt1]))) #reward will be the price change
                    if v > new_v: 
                        new_v = v
                        policy[s] = a

          #Save the best of all actions for the state                                 
                V[s] = new_v
                biggest_change = max(biggest_change,(old_v - V[s]))
            
      #See if the loop should stop now  
        SMALL_ENOUGH = 0.01       
        if biggest_change < SMALL_ENOUGH:
            return V
            break
        iteration += 1




# This you need to write after training your MDP, this should just take in the state and return the Action you would perform
# Please do not make use of the returnNext function inside here, that would defeat the purpose of training the model, as it would be known to you!

def Run(state):
    if v[state] == 0:
      return -1
    if( v[state] > 0 and v[state] <= 1):
      return 0
    else:
      return 1
    
    

# This is the main function, you don't need to tamper with it!
def mainRun(iter = 1000):
    initstate = (random.randint(0,9),random.randint(0,9))
    i = 0
    initprice = 100.00
    price = initprice
    balance = 0
    bondbal = 0
    networth = 0
    st = initstate
    while i < iter:
        act = Run(st)
        print(act)
        pricCh, ns = returnNext(st,act)
        price += pricCh
        if act==1:
            balance -= 10*price
            bondbal += 10
        elif act==-1:
            balance += 10*price
            bondbal -= 10
        
        networth = balance + bondbal*price
        st = ns
        print('Your Networth has went from 0 to ',networth)
        i+=1
















In [ ]:
ftr = input('Test : Filename to read from?')
fte = input('Train : Filename to read?')


with open(ftr,'rb') as f:
    Ztr = pickle.load(f)

with open(fte,'rb') as f:
    Zte = pickle.load(f)

with open('correlations.npy','rb') as f:
    corr = np.load(f)

ifTest  = False
v = Train()
print(v)
ifTest = True
mainRun()

Test : Filename to read from?test.pic
Train : Filename to read?train.pic


In [ ]:
import pandas
keys = v.keys()


In [ ]:
a = np.zeros(shape = (10,10))
for i in keys:
  x,y = i
  a[i[0],i[1]] = v[x,y]

In [ ]:
vdf = pandas.DataFrame(a)

In [ ]:
vdf